In [1]:
import tkinter as tk
from tkinter import messagebox
import time

class SudokuGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sudoku Solver")
        self.root.geometry("500x600")
        
        # Game board (0 represents empty cells)
        self.board = [
            [5, 3, 0, 0, 7, 0, 0, 0, 0],
            [6, 0, 0, 1, 9, 5, 0, 0, 0],
            [0, 9, 8, 0, 0, 0, 0, 6, 0],
            [8, 0, 0, 0, 6, 0, 0, 0, 3],
            [4, 0, 0, 8, 0, 3, 0, 0, 1],
            [7, 0, 0, 0, 2, 0, 0, 0, 6],
            [0, 6, 0, 0, 0, 0, 2, 8, 0],
            [0, 0, 0, 4, 1, 9, 0, 0, 5],
            [0, 0, 0, 0, 8, 0, 0, 7, 9]
        ]
        
        # Solution board
        self.solution = [[0 for _ in range(9)] for _ in range(9)]
        
        # Create UI
        self.create_widgets()
        
        # Copy initial board to solution
        self.copy_board()
        
        # Draw the initial board
        self.draw_board()
    
    def create_widgets(self):
        """Create all GUI elements"""
        # Title label
        tk.Label(self.root, text="SUDOKU SOLVER", font=("Arial", 20, "bold")).pack(pady=10)
        
        # Canvas for drawing the board
        self.canvas = tk.Canvas(self.root, width=450, height=450, bg="white")
        self.canvas.pack()
        
        # Control buttons
        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        
        tk.Button(
            button_frame, 
            text="Solve", 
            command=self.solve_puzzle,
            font=("Arial", 12),
            bg="#4CAF50",
            fg="white"
        ).pack(side=tk.LEFT, padx=5)
        
        tk.Button(
            button_frame, 
            text="Clear", 
            command=self.clear_board,
            font=("Arial", 12),
            bg="#f44336",
            fg="white"
        ).pack(side=tk.LEFT, padx=5)
        
        tk.Button(
            button_frame, 
            text="New Puzzle", 
            command=self.new_puzzle,
            font=("Arial", 12),
            bg="#2196F3",
            fg="white"
        ).pack(side=tk.LEFT, padx=5)
        
        # Status label
        self.status_label = tk.Label(
            self.root, 
            text="Enter numbers and click Solve", 
            font=("Arial", 12)
        )
        self.status_label.pack(pady=5)
    
    def draw_board(self):
        """Draw the Sudoku board on canvas"""
        self.canvas.delete("all")
        
        # Draw grid lines
        for i in range(10):
            width = 3 if i % 3 == 0 else 1
            self.canvas.create_line(
                50 * i, 0, 
                50 * i, 450, 
                width=width
            )
            self.canvas.create_line(
                0, 50 * i, 
                450, 50 * i, 
                width=width
            )
        
        # Draw numbers
        for row in range(9):
            for col in range(9):
                if self.board[row][col] != 0:
                    x = col * 50 + 25
                    y = row * 50 + 25
                    self.canvas.create_text(
                        x, y,
                        text=str(self.board[row][col]),
                        font=("Arial", 20, "bold"),
                        fill="black"
                    )
    
    def copy_board(self):
        """Copy the current board to solution"""
        for i in range(9):
            for j in range(9):
                self.solution[i][j] = self.board[i][j]
    
    def is_valid(self, row, col, num):
        """Check if a number can be placed in a cell"""
        # Check row
        for x in range(9):
            if self.solution[row][x] == num:
                return False
        
        # Check column
        for x in range(9):
            if self.solution[x][col] == num:
                return False
        
        # Check 3x3 box
        box_row = row - row % 3
        box_col = col - col % 3
        for i in range(3):
            for j in range(3):
                if self.solution[box_row + i][box_col + j] == num:
                    return False
        
        return True
    
    def solve(self):
        """Solve the puzzle using backtracking"""
        for row in range(9):
            for col in range(9):
                if self.solution[row][col] == 0:
                    for num in range(1, 10):
                        if self.is_valid(row, col, num):
                            self.solution[row][col] = num
                            
                            # Visualize the solving process
                            self.visualize_move(row, col, num)
                            
                            if self.solve():
                                return True
                            
                            # Backtrack
                            self.solution[row][col] = 0
                            self.visualize_move(row, col, 0)
                    
                    return False
        return True
    
    def visualize_move(self, row, col, num):
        """Visualize the solving process"""
        x = col * 50 + 25
        y = row * 50 + 25
        
        self.canvas.delete("numbers")
        for i in range(9):
            for j in range(9):
                if self.solution[i][j] != 0:
                    x_pos = j * 50 + 25
                    y_pos = i * 50 + 25
                    fill_color = "blue" if self.board[i][j] == 0 else "black"
                    self.canvas.create_text(
                        x_pos, y_pos,
                        text=str(self.solution[i][j]),
                        font=("Arial", 20, "bold"),
                        fill=fill_color,
                        tags="numbers"
                    )
        
        self.root.update()
        time.sleep(0.01)  # Small delay for visualization
    
    def solve_puzzle(self):
        """Solve the puzzle and display solution"""
        self.copy_board()
        self.status_label.config(text="Solving...")
        
        if self.solve():
            self.status_label.config(text="Solution found!")
            self.draw_solution()
        else:
            messagebox.showinfo("No Solution", "This puzzle has no solution!")
            self.status_label.config(text="No solution found")
    
    def draw_solution(self):
        """Draw the solved puzzle"""
        for row in range(9):
            for col in range(9):
                if self.board[row][col] == 0:
                    x = col * 50 + 25
                    y = row * 50 + 25
                    self.canvas.create_text(
                        x, y,
                        text=str(self.solution[row][col]),
                        font=("Arial", 20, "bold"),
                        fill="blue",
                        tags="solution"
                    )
    
    def clear_board(self):
        """Clear the board"""
        for row in range(9):
            for col in range(9):
                self.board[row][col] = 0
        self.draw_board()
        self.status_label.config(text="Board cleared. Enter new numbers.")
    
    def new_puzzle(self):
        """Load a new puzzle"""
        # This is a sample puzzle - you can add more or implement puzzle generation
        self.board = [
            [0, 0, 0, 2, 6, 0, 7, 0, 1],
            [6, 8, 0, 0, 7, 0, 0, 9, 0],
            [1, 9, 0, 0, 0, 4, 5, 0, 0],
            [8, 2, 0, 1, 0, 0, 0, 4, 0],
            [0, 0, 4, 6, 0, 2, 9, 0, 0],
            [0, 5, 0, 0, 0, 3, 0, 2, 8],
            [0, 0, 9, 3, 0, 0, 0, 7, 4],
            [0, 4, 0, 0, 5, 0, 0, 3, 6],
            [7, 0, 3, 0, 1, 8, 0, 0, 0]
        ]
        self.draw_board()
        self.status_label.config(text="New puzzle loaded. Click Solve.")

# Create and run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = SudokuGUI(root)
    root.mainloop()